# Predicting latEnd in LocationDataset

### Notebook automatically generated from your model

Model Lasso (L1) regression (s1), trained on 2023-11-19 08:40:31.

#### Generated on 2023-11-19 13:40:46.145264

prediction
This notebook will reproduce the steps for a REGRESSION on  LocationDataset.
The main objective is to predict the variable latEnd

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [1]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [2]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [3]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {'columns': [{'name': 'ID', 'type': 'bigint'}, {'name': 'eventTimeStart', 'type': 'string'}, {'name': 'eventTimeEnd', 'type': 'string'}, {'name': 'latStart', 'type': 'double'}, {'name': 'lonStart', 'type': 'double'}, {'name': 'latEnd', 'type': 'double'}, {'name': 'lonEnd', 'type': 'double'}], 'userModified': False}

ml_dataset_handle = dataiku.Dataset('LocationDataset')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

CPU times: user 12.6 ms, sys: 4.25 ms, total: 16.9 ms
Wall time: 44.1 ms
Base data has 1537 rows and 7 columns


ID       eventTimeStart         eventTimeEnd   latStart  lonStart     latEnd    lonEnd
0   162  2017-03-22 16:50:25  2017-03-22 17:14:26  38.773537 -9.168737  38.765220 -9.098054
1   136  2017-05-24 12:20:56  2017-05-24 12:21:02  47.409291  8.546942  47.423743  8.555213
2   589  2017-05-24 12:21:08  2017-05-24 12:21:13  47.409291  8.546942  47.423743  8.555213
3  1505  2017-05-24 12:21:17  2017-05-24 12:21:21  47.409291  8.546942  47.423743  8.555213
4   524  2017-05-24 12:21:24  2017-05-24 12:21:31  47.409291  8.546942  47.423743  8.555213

#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [4]:
ml_dataset = ml_dataset[['latEnd', 'eventTimeEnd', 'latStart', 'lonStart', 'eventTimeStart']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [5]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = ['eventTimeEnd', 'eventTimeStart']
numerical_features = ['latStart', 'lonStart']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]') or (hasattr(ml_dataset[feature].dtype, 'base') and ml_dataset[feature].dtype.base == np.dtype('M8[ns]')):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

We renamed the target variable to a column named target

In [6]:
ml_dataset['__target__'] = ml_dataset['latEnd']
del ml_dataset['latEnd']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

#### Cross-validation strategy

The dataset needs to be split into 2 new sets, one that will be used for training the model (train set)
and another that will be used to test its generalization capability (test set)

This is a simple cross-validation strategy.

In [7]:
train, test = pdu.split_train_valid(ml_dataset, prop=0.8)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

Train data has 1229 rows and 5 columns
Test data has 308 rows and 5 columns


#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [8]:
drop_rows_when_missing = []
impute_when_missing = [{'feature': 'latStart', 'impute_with': 'MEAN'}, {'feature': 'lonStart', 'impute_with': 'MEAN'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    test = test[test[feature].notnull()]
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    test[feature['feature']] = test[feature['feature']].fillna(v)
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

Imputed missing values in feature latStart with value 47.1859087778697
Imputed missing values in feature lonStart with value 6.9133492115540855


/home/dataiku/dataiku-dss-12.2.2/python36.packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


We can now handle the categorical features (still using the settings defined in Models):

Let's dummy-encode the following features.
A binary column is created for each of the 100 most frequent values.

In [9]:
LIMIT_DUMMIES = 100

categorical_to_dummy_encode = ['eventTimeEnd', 'eventTimeStart']

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in categorical_to_dummy_encode:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(train, categorical_to_dummy_encode)

def dummy_encode_dataframe(df):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, coerce_to_unicode(dummy_value))
            df[dummy_name] = (df[feature] == dummy_value).astype(float)
        del df[feature]
        print ('Dummy-encoded feature %s' % feature)

dummy_encode_dataframe(train)

dummy_encode_dataframe(test)

Dummy-encoded feature eventTimeEnd
Dummy-encoded feature eventTimeStart


/home/dataiku/dataiku-dss-12.2.2/python36.packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Dummy-encoded feature eventTimeEnd
Dummy-encoded feature eventTimeStart


Let's rescale numerical features

In [10]:
rescale_features = {'latStart': 'AVGSTD', 'lonStart': 'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale
        test[feature_name] = (test[feature_name] - shift).astype(np.float64) / scale

Rescaled latStart
Rescaled lonStart


/home/dataiku/dataiku-dss-12.2.2/python36.packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#### Modeling

Before actually creating our model, we need to split the datasets into their features and labels parts:

In [11]:
X_train = train.drop('__target__', axis=1)
X_test = test.drop('__target__', axis=1)

y_train = np.array(train['__target__'])
y_test = np.array(test['__target__'])

Now we can finally create our model!

In [12]:
from sklearn.linear_model import LassoLarsIC
clf = LassoLarsIC(fit_intercept=True, normalize=True, copy_X=True)

... And train the model

In [13]:
%time clf.fit(X_train, y_train)

CPU times: user 20.3 ms, sys: 13 ms, total: 33.3 ms
Wall time: 11.9 ms


LassoLarsIC(copy_X=True, criterion='aic', eps=2.220446049250313e-16,
      fit_intercept=True, max_iter=500, normalize=True, positive=False,
      precompute='auto', verbose=False)

Build up our result dataset

In [14]:
%time _predictions = clf.predict(X_test)
predictions = pd.Series(data=_predictions, index=X_test.index, name='predicted_value')

# Build scored dataset
results_test = X_test.join(predictions, how='left')
results_test = results_test.join(test['__target__'], how='left')
results_test = results_test.rename(columns= {'__target__': 'latEnd'})

CPU times: user 3.49 ms, sys: 5.11 ms, total: 8.6 ms
Wall time: 2.36 ms


#### Results

You can measure the model's accuracy:

In [15]:
c =  results_test[['predicted_value', 'latEnd']].corr()
print ('Pearson correlation: %s' % c['predicted_value'][1])

Pearson correlation: 0.9935321255058449


That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
